In [33]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [34]:
using LinearAlgebra
using QPControl
using QPControl.Trajectories
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using PushRecovery
using PlanarConvexHulls
BLAS.set_num_threads(1)

In [35]:
@time AtlasRobot.mechanism(add_flat_ground=true);

  0.003963 seconds (29.56 k allocations: 1.467 MiB)


In [36]:
@time mechanism = AtlasRobot.mechanism(add_flat_ground=true);

  0.004749 seconds (29.56 k allocations: 1.534 MiB)


In [37]:
# static contact information
soleframes = add_sole_frames!(mechanism)
foot_polygons = make_foot_polygons(mechanism, soleframes; num_extreme_points=4);

In [38]:
# body lookup
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")

RigidBody: "pelvis"

In [39]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = point.model.friction.μ
            contact = addcontact!(lowlevel, body, position, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [40]:
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)

MechanismState{Float64, Float64, Float64, …}(…)

In [41]:
# ICP stuff
zdes = center_of_mass(nominalstate).v[3] - 0.05
gz = norm(mechanism.gravitational_acceleration)
ω = sqrt(gz / zdes)
icptraj = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-6)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-8)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 10000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    max_polygon_sides = 8 # TODO
    num_segments = 1
    ICPTrajectoryGenerator{Float64, max_polygon_sides}(optimizer, num_segments, ω)
end
# icptraj = let
#     bodyid = BodyID(first(feet))
#     foot_polygon_sole = foot_polygons[bodyid]
#     foot_polygon_world = transform(foot_polygon_sole, transform_to_root(nominalstate, foot_polygon_sole.frame))
#     initial_icp = horizontal_projection(icp(nominalstate, ω).v)
#     final_icp = unwrap(@framechecked(centroid(foot_polygon_world)))
#     f = fit_quintic(x0=0.0, xf=1.0, y0=0.0, yd0=0.0, ydd0=0.0, yf=1.0, ydf=0.0, yddf=0.0)
#     traj = Interpolated(1.0, 3.0, initial_icp, final_icp, f, min_num_derivs=Val(1))
# #     x -> (ret = traj(x, Val(1)); @show x, ret; ret)
#     x -> traj(x, Val(1))
# end
linear_momentum_controller = ICPController(mechanism, icptraj, zdes);

In [42]:
# create high level controller
controller = PushRecoveryController(lowlevel, feet, pelvis, nominalstate, linear_momentum_controller);

In [43]:
# initialize ICP trajectory
gz = norm(mechanism.gravitational_acceleration)
transfer_weight!(icptraj, nominalstate, foot_polygons, BodyID(first(feet)); Δt=5.0);

cop_piece(generator, 1) = PushRecovery.BezierCurves.BezierCurve{3,SArray{Tuple{2},Float64,1,2}}(([0.0355528, -0.0504651], [-0.00391222, 0.0295858], [-0.0451339, 0.1132]))


In [44]:
# create visualizer
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

if !(@isdefined gui) || !any(isopen, MeshCatMechanisms.visualizer(gui.visualizer).core.scope.pool.connections)
    pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0))
    link_colors = Dict(map(foot -> string(foot) => RGBA(0.7f0, 0.7f0, 0.7f0, 0.3f0), feet))
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path=[AtlasRobot.packagepath()], link_colors=link_colors)
    vis = Visualizer()
    mvis = MechanismVisualizer(mechanism, visuals, vis)
    pvis = PushRecoveryVisualizer(mvis)
    gui = GUI(pvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
    copyto!(gui.visualizer, nominalstate)
end

MeshCat Visualizer with path /meshcat/icp

In [45]:
# create ODEProblem
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, pcontroller)
# dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., 10.0); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [46]:
# simulate
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
last(sol.t)

 10.169044 seconds (11.43 M allocations: 719.015 MiB, 5.06% gc time)


10.0

In [47]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult